# 1. Dataset Loading & Synchronization

In [3]:
import os, re
import scipy.io as sio
import pandas as pd
import numpy as np
import cv2
from pathlib import Path

# ----------- Helper: Parse Filenames -----------
def parse_filename(fname):
    """
    Works for files like:
    a10_s1_t1_inertial.mat
    a10_s1_t1_color.avi
    """
    base = os.path.splitext(fname)[0]
    match = re.match(r"a(\d+)_s(\d+)_t(\d+)", base)
    if match:
        activity = int(match.group(1))
        subject = int(match.group(2))
        trial = int(match.group(3))
        return activity, subject, trial
    else:
        raise ValueError(f"Unexpected filename format: {fname}")

# ----------- Load Inertial Data -----------
def load_inertial_dataset(inertial_dir):
    all_data = []
    for file in sorted(os.listdir(inertial_dir)):
        if file.endswith(".mat") and "_inertial" in file:
            data = sio.loadmat(os.path.join(inertial_dir, file))
            signals = data['d_iner']

            # handle shape automatically
            if signals.shape[0] == 6:
                df = pd.DataFrame(signals.T, columns=['acc_x','acc_y','acc_z','gyro_x','gyro_y','gyro_z'])
            elif signals.shape[1] == 6:
                df = pd.DataFrame(signals, columns=['acc_x','acc_y','acc_z','gyro_x','gyro_y','gyro_z'])
            else:
                raise ValueError(f"Unexpected shape {signals.shape} in {file}")

            activity, subject, trial = parse_filename(file)
            df['activity'], df['subject'], df['trial'] = activity, subject, trial
            df['file'] = file
            all_data.append(df)
    return pd.concat(all_data, ignore_index=True)

# ----------- List RGB Videos -----------
def list_rgb_videos(rgb_dir):
    entries = []
    for file in sorted(os.listdir(rgb_dir)):
        if file.endswith(".avi") and "_color" in file:
            a, s, t = parse_filename(file)
            entries.append({
                'activity': a, 'subject': s, 'trial': t,
                'video_path': os.path.join(rgb_dir, file)
            })
    return pd.DataFrame(entries)

# ----------- Synchronize -----------
def synchronize_modalities(inertial_dir, rgb_dir):
    inertial_df = load_inertial_dataset(inertial_dir)
    rgb_index = list_rgb_videos(rgb_dir)

    unique_trials = inertial_df[['activity','subject','trial']].drop_duplicates()
    sync_index = unique_trials.merge(rgb_index, on=['activity','subject','trial'], how='inner')
    return inertial_df, sync_index

DATA_DIR = Path("/content/drive/MyDrive/Mobile Computing Project")
RGB_DIR = DATA_DIR / "RGB"
IMU_DIR = DATA_DIR / "inertial"  # Accelerometer+Gyro files

inertial_df, sync_index = synchronize_modalities(IMU_DIR, RGB_DIR)
print(f"Loaded inertial samples: {inertial_df['file'].nunique()}")
print(f"Synchronized trials: {len(sync_index)}")


Loaded inertial samples: 861
Synchronized trials: 861


# 2. INERTIAL FEATURE EXTRACTION

- compute per-trial features (mean, std, energy, correlations).

In [4]:
from scipy.stats import entropy

def extract_inertial_features(df):
    feats = {}
    for col in ['acc_x','acc_y','acc_z','gyro_x','gyro_y','gyro_z']:
        data = df[col].values
        feats[f'{col}_mean'] = np.mean(data)
        feats[f'{col}_std']  = np.std(data)
        feats[f'{col}_energy'] = np.sum(data**2)/len(data)
        feats[f'{col}_entropy'] = entropy(np.abs(np.fft.fft(data))**2 + 1e-8)
    # correlations between acc axes
    feats['corr_acc_xy'] = np.corrcoef(df['acc_x'], df['acc_y'])[0,1]
    feats['corr_acc_xz'] = np.corrcoef(df['acc_x'], df['acc_z'])[0,1]
    feats['corr_acc_yz'] = np.corrcoef(df['acc_y'], df['acc_z'])[0,1]
    return feats

# Aggregate features per trial
def aggregate_inertial_features(inertial_df):
    feature_list = []
    for (a,s,t), subdf in inertial_df.groupby(['activity','subject','trial']):
        f = extract_inertial_features(subdf)
        f['activity'], f['subject'], f['trial'] = a,s,t
        feature_list.append(f)
    return pd.DataFrame(feature_list)

inertial_features = aggregate_inertial_features(inertial_df)
print(inertial_features.shape)
inertial_features.head()


(861, 30)


,acc_x_mean,acc_x_std,acc_x_energy,acc_x_entropy,acc_y_mean,acc_y_std,acc_y_energy,acc_y_entropy,acc_z_mean,acc_z_std,...,gyro_z_mean,gyro_z_std,gyro_z_energy,gyro_z_entropy,corr_acc_xy,corr_acc_xz,corr_acc_yz,activity,subject,trial
0,-0.556857,0.447787,0.510603,1.170652,-0.091332,0.458671,0.218721,1.984893,-0.550586,0.290265,...,-26.949618,79.318557,7017.715404,1.869359,0.491216,-0.352190,-0.131714,1,1,1
1,-0.549239,0.425393,0.482623,1.172464,-0.113056,0.528621,0.292222,1.991990,-0.538771,0.221657,...,-31.442649,82.787755,7842.452585,1.862082,0.450101,-0.339496,0.183027,1,1,2
2,-0.532076,0.425614,0.464252,1.185028,-0.122434,0.514555,0.279757,2.092485,-0.532993,0.263383,...,-25.077770,78.180118,6741.025337,1.764784,0.378804,-0.489366,0.083605,1,1,3
3,-0.554927,0.445519,0.506431,1.200615,-0.076866,0.502434,0.258348,1.962537,-0.533010,0.266292,...,-25.904339,79.853146,7047.559660,1.777133,0.431769,-0.416411,0.082167,1,1,4
4,-0.554144,0.492936,0.550062,1.374204,-0.231490,0.743558,0.606466,1.879019,-0.108245,0.351101,...,-32.865498,125.991563,16954.014816,1.447494,0.573280,-0.715617,-0.501184,1,2,1


# 3. VISUAL (RGB) FEATURE EXTRACTION

- Sample a few frames from each video

- Extract color histogram + simple motion energy features.

In [5]:
def extract_rgb_features(video_path, frame_sample_rate=10):
    cap = cv2.VideoCapture(video_path)
    frame_count, hist_list, diff_energy = 0, [], []

    ret, prev_frame = cap.read()
    while ret:
        if frame_count % frame_sample_rate == 0:
            gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
            hist = cv2.calcHist([gray],[0],None,[32],[0,256]).flatten()
            hist_list.append(hist)
        ret, frame = cap.read()
        if ret and prev_frame is not None:
            diff = cv2.absdiff(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY),
                               cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY))
            diff_energy.append(np.sum(diff))
            prev_frame = frame
        frame_count += 1
    cap.release()

    if len(hist_list)==0: return None
    hist_mean = np.mean(hist_list, axis=0)
    motion_energy = np.mean(diff_energy) if len(diff_energy)>0 else 0

    features = {f'hist_{i}': hist_mean[i] for i in range(len(hist_mean))}
    features['motion_energy'] = motion_energy
    return features


In [6]:
video_features = []
for _, row in sync_index.iterrows():
    feats = extract_rgb_features(row['video_path'])
    if feats is not None:
        feats['activity'], feats['subject'], feats['trial'] = row['activity'], row['subject'], row['trial']
        video_features.append(feats)

rgb_features = pd.DataFrame(video_features)
print(rgb_features.shape)


(861, 36)


# 4. FEATURE FUSION
- Now, join inertial + RGB features on (activity, subject, trial).

In [7]:
combined_features = inertial_features.merge(
    rgb_features, on=['activity','subject','trial'], how='inner'
).fillna(0)

print(combined_features.shape)
print(combined_features.columns[:10])


(861, 63)
Index(['acc_x_mean', 'acc_x_std', 'acc_x_energy', 'acc_x_entropy',
       'acc_y_mean', 'acc_y_std', 'acc_y_energy', 'acc_y_entropy',
       'acc_z_mean', 'acc_z_std'],
      dtype='object')


# 5. MODEL TRAINING & EVALUATION
- We’ll train a simple RandomForestClassifier and evaluate accuracy per activity.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Split
X = combined_features.drop(columns=['activity','subject','trial'])
y = combined_features['activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

# Train
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9722222222222222
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00         8
           3       0.89      1.00      0.94         8
           4       1.00      1.00      1.00         8
           5       1.00      1.00      1.00         8
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00         8
           8       1.00      1.00      1.00         8
           9       1.00      1.00      1.00         8
          10       1.00      0.88      0.93         8
          11       0.89      1.00      0.94         8
          12       1.00      1.00      1.00         8
          13       0.89      1.00      0.94         8
          14       1.00      1.00      1.00         8
          15       1.00      1.00      1.00         8
          16       1.00      1.00      1.00         8
          17       1.00      0.88      0.93         

# Compare Modalities Individually
- To show contribution of each modality

In [9]:
# Inertial-only
X_i = inertial_features.drop(columns=['activity','subject','trial'])
y_i = inertial_features['activity']
Xi_train, Xi_test, yi_train, yi_test = train_test_split(X_i, y_i, test_size=0.25, stratify=y_i, random_state=42)
clf.fit(Xi_train, yi_train)
print("Inertial-only Accuracy:", accuracy_score(yi_test, clf.predict(Xi_test)))

# RGB-only
X_v = rgb_features.drop(columns=['activity','subject','trial'])
y_v = rgb_features['activity']
Xv_train, Xv_test, yv_train, yv_test = train_test_split(X_v, y_v, test_size=0.25, stratify=y_v, random_state=42)
clf.fit(Xv_train, yv_train)
print("RGB-only Accuracy:", accuracy_score(yv_test, clf.predict(Xv_test)))


Inertial-only Accuracy: 0.9583333333333334
RGB-only Accuracy: 0.7407407407407407
